In [1]:
model_path = "C:\\Users\\subin\\.cache\\huggingface\\hub\\models--sentence-transformers--all-MiniLM-L6-v2\\snapshots\\all-MiniLM-L6-v2"

In [2]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer(model_path)

d:\My Projects\Tiny Llama\finetune.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
statement1 = model.encode('joy')

In [4]:
statement2 = model.encode('sadness')

In [5]:
sim = util.cos_sim(statement1, statement2)

In [6]:
sim

tensor([[0.4299]])

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "C:\\Users\\subin\\.cache\\huggingface\\hub\\models--SamLowe--roberta-base-go_emotions\\snapshots\\roberta-goemo"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

d:\My Projects\CAPSTONE RESEARCH\finetune.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [137]:
def get_classification(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs

txt = "I love the book"
pred = get_classification(txt, model, tokenizer)
pred

SequenceClassifierOutput(loss=None, logits=tensor([[-2.6461, -5.8056, -5.7108, -5.4754, -3.9026, -5.8223, -5.8843, -6.3264,
         -5.5088, -5.5651, -5.4251, -6.2885, -7.5970, -5.6135, -7.1344, -4.9963,
         -7.9100, -4.5098,  2.8664, -7.6593, -5.1790, -7.7158, -5.3941, -7.9463,
         -6.6800, -5.6090, -6.3140, -4.6681]]), hidden_states=None, attentions=None)

In [ ]:
# SIMILARITY TEST

import torch
from torch.nn.functional import cosine_similarity
import numpy as np

def get_sentence_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]

        # Mean pooling
        attention_mask = inputs['attention_mask']
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        #sentence_embedding = sum_embeddings / sum_mask

    return mask_expanded

s1 = "I love my father"
#e1 = 'love'
#
#s2 = "I lost my mom, RIP"
#e2 = 'sadness'


s1_emb = get_sentence_embedding(s1, model, tokenizer)
#e1_emb = get_sentence_embedding(e1, model, tokenizer)
#s2_emb = get_sentence_embedding(s2, model, tokenizer)
#e2_emb = get_sentence_embedding(s2, model, tokenizer)

In [9]:
s1_emb.shape

torch.Size([1, 6, 768])

In [8]:
c1 = torch.cat((s1_emb, e1_emb), dim=-1)
c2 = torch.cat((s2_emb, e2_emb), dim=-1)

cosine_similarity(c1, c2).item()

0.2503473460674286

In [12]:
# SIMILARITY TEST

import torch
from torch.nn.functional import cosine_similarity
import numpy as np

def get_sentence_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]

        # Mean pooling
        attention_mask = inputs['attention_mask']
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        sentence_embedding = sum_embeddings / sum_mask

    return sentence_embedding

s1 = "satisfied with one's achievement or high standards"
s2 = "feeling of worry or distress being lifted"


s1_emb = get_sentence_embedding(s1, model, tokenizer)
s2_emb = get_sentence_embedding(s2, model, tokenizer)

In [13]:
cosine_similarity(s1_emb, s2_emb).item()

0.26255878806114197

In [ ]:
0.2658894658088684

0.22648681700229645

In [ ]:
0.11941459774971008

### CLS Embedding Fails

In [128]:
# CLS TESTING

text1 = "I lost my mom, RIP"

inputs = tokenizer(text1, return_tensors='pt')

with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)

last_hidden_state = outputs.hidden_states[-1]

cls_embedding_text1 = last_hidden_state[:, 0, :]

In [23]:
cls_embedding_text1.shape

torch.Size([1, 768])

In [116]:
E1 = cls_embedding_text1*cls_embedding_emotion1

In [115]:
from torch.nn.functional import cosine_similarity

cosine_similarity(cls_embedding_text1, cls_embedding_emotion1).item()

0.9417742490768433

In [129]:
# CLS TESTING

text2 = "I love my father"

inputs = tokenizer(text1, return_tensors='pt')

with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)

last_hidden_state = outputs.hidden_states[-1]

cls_embedding_text2 = last_hidden_state[:, 0, :]


In [130]:
cosine_similarity(cls_embedding_text1, cls_embedding_text2).item()

0.9999999403953552

In [117]:
E2 = cls_embedding_text2*cls_embedding_emotion2

In [118]:
cosine_similarity(E1, E5).item()

0.378205269575119

# RoBERTa-base

In [1]:
from transformers import AutoTokenizer, AutoModel

d:\My Projects\CAPSTONE RESEARCH\finetune.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModel

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModel.from_pretrained("roberta-base")

d:\My Projects\CAPSTONE RESEARCH\finetune.env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\subin\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP d

In [3]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr